# cafe sales time series analysisThis notebook loads data and prepares the data for cafe sales time series analysis .

In [ ]:
import pandas as pdimport numpy as npprint('Dataset is loaded for analysis')

In [ ]:
# load cafe sales dataset df = pd.read_csv('dirty_cafe_sales.csv') df.head()